<a href="https://colab.research.google.com/github/knoppin1/DATA-606/blob/main/merge_income_vax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import pickle

In [ ]:
vax_df = pd.read_pickle("./vaccinations.pkl")
vax_df.shape

(3224, 6)

In [ ]:
vax_df.head()

,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
0,2021-06-11,13265,GA,Taliaferro County,130,8.5
1,2021-06-11,28083,MS,Leflore County,9114,32.3
2,2021-06-11,37103,NC,Jones County,3058,32.5
3,2021-06-11,48491,TX,Williamson County,0,0.0
4,2021-06-11,27099,MN,Mower County,17800,44.4


In [ ]:
vax_df = vax_df.sort_values(by=['State','County'], ascending=[True,True])
vax_df.head()

,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
3028,2021-06-11,2013,AK,Aleutians East Borough,2134,63.9
55,2021-06-11,2016,AK,Aleutians West Census Area,2420,43.0
1987,2021-06-11,2020,AK,Anchorage Municipality,122801,42.6
739,2021-06-11,2050,AK,Bethel Census Area,8739,47.5
2447,2021-06-11,2060,AK,Bristol Bay Borough,619,74.0


In [ ]:
incomes_df = pd.read_pickle("./incomes.pkl")
incomes_df.shape

(3112, 3)

In [ ]:
incomes_df.head()

,State,County,Income
0,Alabama,Autauga,43917
1,Alabama,Baldwin,47485
2,Alabama,Barbour,35763
3,Alabama,Bibb,31725
4,Alabama,Blount,36412


In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
    

In [ ]:
def state_abbreviate(state):
    return us_state_abbrev[state]

In [ ]:
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware',
          'District of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana',
          'Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',
          'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire',
          'New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma',
          'Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee',
          'Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [ ]:
state_abbreviations = ['AL','AK','AS','AZ','AR','CA','CO','CT',
                       'DE','DC','FL','GA','HI','ID','IL','IN',
                       'IA','KS','KY','LA','ME','MD','MA','MI',
                       'MN','MS','MO','MT','NE','NV','NH','NJ',
                       'NM','NY','NC','ND','OH','OK','OR','PA',
                       'RI','SC','SD','TN','TX','UT','VT','VA',
                       'WA','WV','WI','WY']

Convert State names to abbreviations.

In [ ]:
incomes_df['State'] = incomes_df['State'].apply(state_abbreviate)
incomes_df = incomes_df.sort_values(by=['State','County'], ascending=[True,True])
incomes_df.head()

,State,County,Income
67,AK,Aleutians East Borough,62537
68,AK,Aleutians West Census Area,58008
69,AK,Anchorage Municipality,70145
70,AK,Bethel Census Area,46348
71,AK,Bristol Bay Borough,151900


In [ ]:
incomes_df.reset_index(inplace=True)
vax_df.reset_index(inplace=True)

Need to join vaccinations and incomes based on County names, so need to address differences.

In [ ]:
incomes_df.County[-10:]

3102      Niobrara
3103          Park
3104        Platte
3105      Sheridan
3106      Sublette
3107    Sweetwater
3108         Teton
3109         Uinta
3110      Washakie
3111        Weston
Name: County, dtype: object

In [ ]:
vax_df.County[-10:]

3214      Niobrara County
3215          Park County
3216        Platte County
3217      Sheridan County
3218      Sublette County
3219    Sweetwater County
3220         Teton County
3221         Uinta County
3222      Washakie County
3223        Weston County
Name: County, dtype: object

Drop 'County' portion of county name from vaccine dataframe County column.

In [ ]:
vax_df.County = vax_df.County.replace(regex=[' County', ' Parish'], value = '')
vax_df.County[-10:]

3214      Niobrara
3215          Park
3216        Platte
3217      Sheridan
3218      Sublette
3219    Sweetwater
3220         Teton
3221         Uinta
3222      Washakie
3223        Weston
Name: County, dtype: object

Now check for county names with 'City' or 'city' in the name.

In [ ]:
for county in vax_df['County']:
 if county.__contains__('city') or county.__contains__('City'):
    print(county)

Juneau City and Borough
Sitka City and Borough
Wrangell City and Borough
Yakutat City and Borough
Baltimore city
St. Louis city
Carson City
Alexandria city
Bristol city
Buena Vista city
Charles City
Charlottesville city
Chesapeake city
Colonial Heights city
Covington city
Danville city
Emporia city
Fairfax city
Falls Church city
Franklin city
Fredericksburg city
Galax city
Hampton city
Harrisonburg city
Hopewell city
James City
Lexington city
Lynchburg city
Manassas Park city
Manassas city
Martinsville city
Newport News city
Norfolk city
Norton city
Petersburg city
Poquoson city
Portsmouth city
Radford city
Richmond city
Roanoke city
Salem city
Staunton city
Suffolk city
Virginia Beach city
Waynesboro city
Williamsburg city
Winchester city


In [ ]:
# Check incomes  dataframe.
for county in incomes_df['County']:
 if county.__contains__('City') or county.__contains__('City'):
    print(county)

Juneau City and Borough
Sitka City and Borough
Wrangell City and Borough
Yakutat City and Borough
Baltimore City
St. Louis City
Carson City
Charles City
Fairfax, Fairfax City + Falls Church
James City + Williamsburg


Remove 'city' portion of county name in vaccine dataframe County column.

In [ ]:
vax_df.County = vax_df.County.replace(regex=[' city',' City'], value='')

Remove 'City' portion of county name in incomes dataframe County column.

In [ ]:
incomes_df.County = incomes_df.County.replace(regex=[' City'], value='')

Drop 'City' from county names containing 'Borough' from both dataframes

In [ ]:
incomes_df.County = incomes_df.County.replace(regex=[' (includes Yellowstone National Park)'], value='')

In [ ]:
print('income rows:\t\t',incomes_df.shape[0])
print('vaccination rows:\t',vax_df.shape[0])

income rows:		 3112
vaccination rows:	 3224


Remove rows in vaccination dataframe for non-US states.

In [ ]:
vax_df = vax_df[vax_df['State'].isin(state_abbreviations)].reset_index(drop=True)
vax_df.shape

(3142, 7)

Determine counties in vaccinations dataframe but not in incomes dataframe.

In [ ]:
vax_df[~vax_df.County.isin(incomes_df.County.values)]

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
20,2024,2021-06-11,2195,AK,Petersburg Census Area,1910,58.5
316,183,2021-06-11,11001,DC,District of Columbia,320073,45.4
548,2516,2021-06-11,15005,HI,Kalawao,0,0.0
550,1283,2021-06-11,15009,HI,Maui,0,0.0
839,1081,2021-06-11,18087,IN,LaGrange,6659,16.8
1938,3231,2021-06-11,35013,NM,Dona Ana,81205,37.2
2807,724,2021-06-11,51003,VA,Albemarle,21132,19.3
2814,3087,2021-06-11,51015,VA,Augusta,14601,19.3
2829,1158,2021-06-11,51540,VA,Charlottesville,2466,5.2
2833,2218,2021-06-11,51570,VA,Colonial Heights,108,0.6


Investigate HI counties

In [ ]:
filter = 'State == ' + '"HI"'
incomes_df.query(filter)

,index,State,County,Income
545,545,HI,Hawaii,43578
546,546,HI,Honolulu,61174
547,547,HI,Kauai,51545
548,548,HI,Maui + Kalawao,51348


Need to split counties that have been combined in incomes dataframe.

In [ ]:
for county in incomes_df['County']:
 if county.__contains__('+'):
    print(county)

Maui + Kalawao
Albemarle + Charlottesville
Alleghany + Covington
Augusta, Staunton + Waynesboro
Campbell + Lynchburg
Carroll + Galax
Dinwiddie, Colonial Heights + Petersburg
Fairfax, Fairfax + Falls Church
Frederick + Winchester
Greensville + Emporia
Henry + Martinsville
James + Williamsburg
Montgomery + Radford
Pittsylvania + Danville
Prince George + Hopewell
Prince William, Manassas + Manassas Park
Roanoke + Salem
Rockbridge, Buena Vista + Lexington
Rockingham + Harrisonburg
Southampton + Franklin
Spotsylvania + Fredericksburg
Washington + Bristol
Wise + Norton
York + Poquoson


----
Function: split_comma_plus 

- returns a list of counties that were combined in a single string

In [ ]:
def split_comma_plus(s):
    l = re.split(r",|\+", s)
    l = [x.lstrip(" ") for x in l]
    l = [x.rstrip(" ") for x in l]
    return l

Test split_comma_plus

In [ ]:
split_comma_plus("a, b + c")

['a', 'b', 'c']

In [ ]:
split_comma_plus("b + c")

['b', 'c']

---
Create a new incomes dataframe with new rows for each county that had been combined.  

Note: The median income is the same for each county that was part of a combined record.

In [ ]:
new_incomes_df = pd.DataFrame(columns=['State','County','Income'])
for index, row in incomes_df.iterrows():
    split_list = []
    split_list = split_comma_plus(row.County)
    for county in split_list:
        new_row = row
        new_row.County = county
        new_incomes_df = new_incomes_df.append(new_row, ignore_index=True)

new_incomes_df = new_incomes_df.drop(['index'], axis=1)
new_incomes_df.shape

(3141, 3)

Check VA since it had the most combined counties.

In [ ]:
filter = 'State == ' + '"VA"'
new_incomes_df.query(filter)

,State,County,Income
2805,VA,Accomack,42923
2806,VA,Albemarle,77657
2807,VA,Charlottesville,77657
2808,VA,Alexandria,91990
2809,VA,Alleghany,40928
2810,VA,Covington,40928
2811,VA,Amelia,44297
2812,VA,Amherst,38165
2813,VA,Appomattox,39268
2814,VA,Arlington,99407


Now look for vaccination rows not in the new incomes dataframe.

In [ ]:
vax_df[~vax_df.County.isin(new_incomes_df.County.values)]

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
20,2024,2021-06-11,2195,AK,Petersburg Census Area,1910,58.5
316,183,2021-06-11,11001,DC,District of Columbia,320073,45.4
839,1081,2021-06-11,18087,IN,LaGrange,6659,16.8
1938,3231,2021-06-11,35013,NM,Dona Ana,81205,37.2


In [ ]:
new_incomes_df[~new_incomes_df.County.isin(vax_df.County.values)]

,State,County,Income
20,AK,Petersburg Borough,74021
670,ID,Fremont (includes Yellowstone National Park),37953
839,IN,Lagrange,43275
1937,NM,Doña Ana,37756


Fix remaining discrepancies.

In [ ]:
vax_df.replace("Petersburg Census Area","Petersburg Borough", inplace=True)
new_incomes_df.replace("Fremont (includes Yellowstone National Park)","Fremont", inplace=True)
new_incomes_df.replace("Lagrange","LaGrange", inplace=True)
new_incomes_df.replace("Doña Ana","Dona Ana", inplace=True)

In [ ]:
vax_df[~vax_df.County.isin(new_incomes_df.County.values)]

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
316,183,2021-06-11,11001,DC,District of Columbia,320073,45.4


In [ ]:
new_incomes_df[~new_incomes_df.County.isin(vax_df.County.values)]

,State,County,Income


In [ ]:
vax_df.shape[0]

3142

In [ ]:
new_incomes_df.shape[0]

3141

In [ ]:
merged_df = vax_df.merge(new_incomes_df[['State', 'County', 'Income']], how = 'inner')
merged_df.head()

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct,Income
0,3028,2021-06-11,2013,AK,Aleutians East Borough,2134,63.9,62537
1,55,2021-06-11,2016,AK,Aleutians West Census Area,2420,43.0,58008
2,1987,2021-06-11,2020,AK,Anchorage Municipality,122801,42.6,70145
3,739,2021-06-11,2050,AK,Bethel Census Area,8739,47.5,46348
4,2447,2021-06-11,2060,AK,Bristol Bay Borough,619,74.0,151900


In [ ]:
merged_df.drop(['index'], axis = 1, inplace = True)

In [ ]:
merged_df.shape

(3152, 7)

There are too many rows in the merged dataframe, so check for duplicates.

In [ ]:
dups_df = merged_df.duplicated(subset = ['State','County'],keep=False)
dups_df.sum()

24

In [ ]:
if dups_df.any():
    print(merged_df.loc[dups_df], end='\n\n')

           Date   FIPS State     County  Vaccinated  Vaccinated Pct  Income
1191 2021-06-11  24005    MD  Baltimore      403538            48.8   62976
1192 2021-06-11  24005    MD  Baltimore      403538            48.8   53378
1193 2021-06-11  24510    MD  Baltimore      243362            41.0   62976
1194 2021-06-11  24510    MD  Baltimore      243362            41.0   53378
1501 2021-06-11  29189    MO  St. Louis      398408            40.1   73016
1502 2021-06-11  29189    MO  St. Louis      398408            40.1   48202
1503 2021-06-11  29510    MO  St. Louis      103632            34.5   73016
1504 2021-06-11  29510    MO  St. Louis      103632            34.5   48202
2845 2021-06-11  51059    VA    Fairfax      501811            43.7   86141
2846 2021-06-11  51059    VA    Fairfax      501811            43.7   86141
2847 2021-06-11  51600    VA    Fairfax        1345             5.6   86141
2848 2021-06-11  51600    VA    Fairfax        1345             5.6   86141
2853 2021-06

These duplicates are due to dropping 'City' because these names have both a county and city name.  

Need to fix this, so reread vaccinations pickle and find the correct rows to update.

In [ ]:
vax1_df = pd.read_pickle("./vaccinations.pkl")
vax1_df.shape

(3224, 6)

In [ ]:
filter = "FIPS in "  + "[24005, 24510, 29189, 29510, 51059, 51600, 51067, 51620, 51159, 51760, 51161, 51770]"
hits = vax1_df.query(filter)
hits 

,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
173,2021-06-11,51067,VA,Franklin County,11062,19.7
194,2021-06-11,29189,MO,St. Louis County,398408,40.1
312,2021-06-11,51059,VA,Fairfax County,501811,43.7
391,2021-06-11,51159,VA,Richmond County,662,7.3
831,2021-06-11,51161,VA,Roanoke County,1408,1.5
1084,2021-06-11,51600,VA,Fairfax city,1345,5.6
1277,2021-06-11,51620,VA,Franklin city,0,0.0
1488,2021-06-11,29510,MO,St. Louis city,103632,34.5
2100,2021-06-11,24510,MD,Baltimore city,243362,41.0
2228,2021-06-11,24005,MD,Baltimore County,403538,48.8


In [ ]:
city_hits = hits.County.str.contains('city').to_frame()
city_hits

,County
173,False
194,False
312,False
391,False
831,False
1084,True
1277,True
1488,True
2100,True
2228,False


In [ ]:
city_hits = city_hits[city_hits.County]
hit_list = [h for h in city_hits.index]
hit_list

[1084, 1277, 1488, 2100, 2656, 2830]

In [ ]:
for fips in [51600,51620,29510,24510,51770,51760]:
    filter = "FIPS == " + str(fips)
    indx = vax_df.query(filter).index
    vax_df.iloc[indx, vax_df.columns.get_loc('County')] += ' City'
    print(vax_df.query(filter).County)

2844    Fairfax City
Name: County, dtype: object
2850    Franklin City
Name: County, dtype: object
1502    St. Louis City
Name: County, dtype: object
1194    Baltimore City
Name: County, dtype: object
2913    Roanoke City
Name: County, dtype: object
2911    Richmond City
Name: County, dtype: object


In [ ]:
merged_df = vax_df.merge(new_incomes_df[['State', 'County', 'Income']], how = 'inner')
merged_df.head()

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct,Income
0,3028,2021-06-11,2013,AK,Aleutians East Borough,2134,63.9,62537
1,55,2021-06-11,2016,AK,Aleutians West Census Area,2420,43.0,58008
2,1987,2021-06-11,2020,AK,Anchorage Municipality,122801,42.6,70145
3,739,2021-06-11,2050,AK,Bethel Census Area,8739,47.5,46348
4,2447,2021-06-11,2060,AK,Bristol Bay Borough,619,74.0,151900


In [ ]:
print('vax shape\t',vax_df.shape)
print('incomes shape\t',new_incomes_df.shape)
print('merged shape\t',merged_df.shape)

vax shape	 (3142, 7)
incomes shape	 (3141, 3)
merged shape	 (3140, 8)


In [ ]:
dups_df = merged_df.duplicated(subset = ['State','County'],keep=False)
dups_df.sum()

12

In [ ]:
if dups_df.any():
    print(merged_df.loc[dups_df], end='\n\n')

      index       Date   FIPS State     County  Vaccinated  Vaccinated Pct  \
1191   2228 2021-06-11  24005    MD  Baltimore      403538            48.8   
1192   2228 2021-06-11  24005    MD  Baltimore      403538            48.8   
1499    194 2021-06-11  29189    MO  St. Louis      398408            40.1   
1500    194 2021-06-11  29189    MO  St. Louis      398408            40.1   
2841    312 2021-06-11  51059    VA    Fairfax      501811            43.7   
2842    312 2021-06-11  51059    VA    Fairfax      501811            43.7   
2847    173 2021-06-11  51067    VA   Franklin       11062            19.7   
2848    173 2021-06-11  51067    VA   Franklin       11062            19.7   
2908    391 2021-06-11  51159    VA   Richmond         662             7.3   
2909    391 2021-06-11  51159    VA   Richmond         662             7.3   
2910    831 2021-06-11  51161    VA    Roanoke        1408             1.5   
2911    831 2021-06-11  51161    VA    Roanoke        1408      

In [ ]:
income1_df = pd.read_pickle("./incomes.pkl")
income1_df.shape

(3112, 3)

In [ ]:
l = ['Baltimore','St. Louis','Fairfax','Franklin','Richmond','Roanoke']
    

Fix Baltimore City

In [ ]:
income1_df[(income1_df['County'].str.contains('Baltimore'))]

,State,County,Income
1193,Maryland,Baltimore,62976
1214,Maryland,Baltimore City,53378


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('Baltimore'))]

,State,County,Income
1192,MD,Baltimore,62976
1193,MD,Baltimore,53378


In [ ]:
new_incomes_df.iloc[1193, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('Baltimore'))]

,State,County,Income
1192,MD,Baltimore,62976
1193,MD,Baltimore City,53378


Fix St. Louis City

In [ ]:
income1_df[(income1_df['County'].str.contains('St. Louis'))]

,State,County,Income
1380,Minnesota,St. Louis,48718
1576,Missouri,St. Louis,73016
1595,Missouri,St. Louis City,48202


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('St. Louis'))]

,State,County,Income
1384,MN,St. Louis,48718
1500,MO,St. Louis,73016
1501,MO,St. Louis,48202


In [ ]:
new_incomes_df.iloc[1501, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('St. Louis'))]

,State,County,Income
1384,MN,St. Louis,48718
1500,MO,St. Louis,73016
1501,MO,St. Louis City,48202


Fix Fairfax City

In [ ]:
income1_df[(income1_df['County'].str.contains('Fairfax'))]

,State,County,Income
2906,Virginia,"Fairfax, Fairfax City + Falls Church",86141


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('Fairfax'))]

,State,County,Income
2843,VA,Fairfax,86141
2844,VA,Fairfax,86141


In [ ]:
new_incomes_df.iloc[2844, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('Fairfax'))]

,State,County,Income
2843,VA,Fairfax,86141
2844,VA,Fairfax City,86141


Fix Franklin City

In [ ]:
income1_df[(income1_df['County'].str.contains('Franklin'))]

,State,County,Income
29,Alabama,Franklin,35292
134,Arkansas,Franklin,33404
336,Florida,Franklin,37320
444,Georgia,Franklin,34017
569,Idaho,Franklin,37293
620,Illinois,Franklin,38289
718,Indiana,Franklin,47664
821,Iowa,Franklin,53241
915,Kansas,Franklin,44472
1027,Kentucky,Franklin,43271


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('Franklin'))]

,State,County,Income
58,AL,Franklin,35292
119,AR,Franklin,33404
336,FL,Franklin,37320
444,GA,Franklin,34017
584,IA,Franklin,53241
669,ID,Franklin,37293
720,IL,Franklin,38289
818,IN,Franklin,47664
916,KS,Franklin,44472
1028,KY,Franklin,43271


In [ ]:
new_incomes_df.iloc[2920, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('Franklin'))]

,State,County,Income
58,AL,Franklin,35292
119,AR,Franklin,33404
336,FL,Franklin,37320
444,GA,Franklin,34017
584,IA,Franklin,53241
669,ID,Franklin,37293
720,IL,Franklin,38289
818,IN,Franklin,47664
916,KS,Franklin,44472
1028,KY,Franklin,43271


Fix Richmond

In [ ]:
income1_df[(income1_df['County'].str.contains('Richmond'))]

,State,County,Income
506,Georgia,Richmond,39370
1868,New York,Richmond,58890
1964,North Carolina,Richmond,35630
2878,Virginia,Richmond,37741
2896,Virginia,Richmond,56560


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('Richmond'))]

,State,County,Income
506,GA,Richmond,39370
1729,NC,Richmond,35630
2022,NY,Richmond,58890
2905,VA,Richmond,37741
2906,VA,Richmond,56560


In [ ]:
new_incomes_df.iloc[2906, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('Richmond'))]

,State,County,Income
506,GA,Richmond,39370
1729,NC,Richmond,35630
2022,NY,Richmond,58890
2905,VA,Richmond,37741
2906,VA,Richmond City,56560


In [ ]:
vax_df[(vax_df['County'].str.contains('Richmond'))]

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
507,2795,2021-06-11,13245,GA,Richmond,44613,22.0
1730,3042,2021-06-11,37153,NC,Richmond,13316,29.7
2023,2913,2021-06-11,36085,NY,Richmond,205773,43.2
2910,391,2021-06-11,51159,VA,Richmond,662,7.3
2911,2830,2021-06-11,51760,VA,Richmond City,28338,12.3


Fix Roanoke

In [ ]:
income1_df[(income1_df['County'].str.contains('Roanoke'))]

,State,County,Income
2897,Virginia,Roanoke,45277
2915,Virginia,Roanoke + Salem,53489


In [ ]:
new_incomes_df[(new_incomes_df['County'].str.contains('Roanoke'))]

,State,County,Income
2907,VA,Roanoke,45277
2908,VA,Roanoke,53489


In [ ]:
new_incomes_df.iloc[2907, new_incomes_df.columns.get_loc('County')] += ' City'
new_incomes_df[(new_incomes_df['County'].str.contains('Roanoke'))]

,State,County,Income
2907,VA,Roanoke City,45277
2908,VA,Roanoke,53489


In [ ]:
vax_df[(vax_df['County'].str.contains('Roanoke'))]

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct
2912,831,2021-06-11,51161,VA,Roanoke,1408,1.5
2913,2656,2021-06-11,51770,VA,Roanoke City,6176,6.2


----

Merge again to get a clean combination

In [ ]:
merged_df = vax_df.merge(new_incomes_df[['State', 'County', 'Income']], how = 'inner')
merged_df.head(3)

,index,Date,FIPS,State,County,Vaccinated,Vaccinated Pct,Income
0,3028,2021-06-11,2013,AK,Aleutians East Borough,2134,63.9,62537
1,55,2021-06-11,2016,AK,Aleutians West Census Area,2420,43.0,58008
2,1987,2021-06-11,2020,AK,Anchorage Municipality,122801,42.6,70145


In [ ]:
merged_df = merged_df.drop(['index'],axis=1)
merged_df.head(3)

,Date,FIPS,State,County,Vaccinated,Vaccinated Pct,Income
0,2021-06-11,2013,AK,Aleutians East Borough,2134,63.9,62537
1,2021-06-11,2016,AK,Aleutians West Census Area,2420,43.0,58008
2,2021-06-11,2020,AK,Anchorage Municipality,122801,42.6,70145


In [ ]:
print('vax shape\t',vax_df.shape)
print('incomes shape\t',new_incomes_df.shape)
print('merged shape\t',merged_df.shape)

vax shape	 (3142, 7)
incomes shape	 (3141, 3)
merged shape	 (3140, 7)


In [ ]:
dups_df = merged_df.duplicated(subset = ['State','County'],keep=False)
dups_df.sum()

0

Finally - success!

Save the merged dataframe to a pickle file.

In [ ]:
merged_df.to_pickle("./merged_income_vax.pkl")